In [85]:
import array
import contextlib
import matplotlib
import numpy
import wave

from matplotlib import pyplot, mlab

from scipy.ndimage.filters import maximum_filter
from scipy.ndimage.morphology import generate_binary_structure, binary_erosion, iterate_structure

In [86]:
WINDOW_SIZE = 4096
WINDOW_OVERLAP = 2048

In [87]:
def read_wav(file):
    """Read a Wave file into an array
    :param file: location of the Wave file
    :return: array containing the amplitude at each sample
    """
    with contextlib.closing(wave.open(file)) as f:
        params = f.getparams()
        frames = f.readframes(params[3])
    return array.array("h", frames), params

In [88]:
def get_spectrogram(signal):
    """Run FFT to get the spectrogram of a signal.
    :param signal: array of amplitudes
    :param window_size:
    :param window_overlap:
    :return: numpy 2D array x: time, y: frequency
    """
    result = matplotlib.mlab.specgram(
        signal,
        NFFT=WINDOW_SIZE,
        Fs=44100,
        window=matplotlib.mlab.window_hanning,
        noverlap=WINDOW_OVERLAP
    )[0]

    result = 10 * numpy.log10(result)
    result[result == -numpy.inf] = 0
    return result

In [89]:
def get_peaks(image, plot=False):
    """Get the peaks from the 2D array, and do some filtering to reduce the number of peaks.
    :param image: 2D array representing the image
    :return: List of (x, y)
    """
    # http://stackoverflow.com/questions/3684484/peak-detection-in-a-2d-array
    structure = generate_binary_structure(2, 1)
    neighborhood = iterate_structure(structure, 20)

    local_max = maximum_filter(image, footprint=neighborhood)==image
    background = (image == 0)
    eroded_background = binary_erosion(background, structure=neighborhood, border_value=1)
    detected_peaks = np.bitwise_xor(local_max,eroded_background)

    # detected_peaks is a 2D mask
    amplitudes = image[detected_peaks].flatten()
    freq, t = numpy.where(detected_peaks)

    # Filter all peaks with amplitude less then 10
    unfiltered_peaks = zip(t, freq, amplitudes)
    filtered_peaks = [x for x in unfiltered_peaks if x[2] > 10]

    # get indices for frequency and time
    time = [x[0] for x in filtered_peaks]
    frequency = [x[1] for x in filtered_peaks]

    # scatter of the peaks
    if plot:
        fig, ax = pyplot.subplots()
        ax.imshow(image)
        ax.scatter(time, frequency)
        ax.set_xlabel('Time')
        ax.set_ylabel('Frequency')
        ax.set_title("Spectrogram")
        pyplot.gca().invert_yaxis()
        pyplot.show()

    return list(zip(time, frequency))

In [90]:
def create_fingerprints(peaks):
    hashes = []
    for i in range(len(peaks)):
        for j in range(1, 15):
            if (i + j) < len(peaks):
                t1 = peaks[i][0]
                t2 = peaks[i + j][0]
                freq1 = peaks[i][1]
                freq2 = peaks[i + j][1]
                delta = t2 - t1

                hash = hashlib.sha1(str(str(freq1) + str(freq2) + str(delta)).encode('utf-8'))

                hashes.append((hash.hexdigest(), t1))

    return hashes

In [91]:
data, params = read_wav('Violin.wav')

In [92]:
metadata = {'title':'title', 'author':'author', 'album':'album'}

In [93]:
spec = get_spectrogram(data)

In [94]:
peaks = get_peaks(spec)

In [95]:
fp1 = create_fingerprints(peaks)

In [97]:
fp1

AttributeError: 'list' object has no attribute 'shape'

In [74]:
class Fingerprint(object):
    def __init__(self, hashvalue, timeoffset):
        self.hashvalue = hashvalue
        self.timeoffset = timeoffset

    def __eq__(self, other):
        return self.hashvalue == other.hashvalue

    def __str__(self):
        return "Fingerprint: {0} at offset {1}".format(self.hashvalue, self.timeoffset)

In [75]:
def find_matches(fingerprints_1, fingerprints_2):
    matches = {}

    for fp_1 in fingerprints_1:
        for fp_2 in fingerprints_2:
            if fp_1 == fp_2:
                matches[fp_1.timeoffset - fp_2.timeoffset] = fp_1.hashvalue

    return matches

In [98]:
data2, params2 = read_wav('RECORDING2.wav')

In [99]:
spec2 = get_spectrogram(data2)

In [100]:
peaks2 = get_peaks(spec2)

In [101]:
fp2 = create_fingerprints(peaks2)